https://dacon.io/competitions/official/235864/codeshare/4229?page=1&dtype=recent


In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_excel('news_label.xlsx', engine='openpyxl')

In [3]:
df = df.rename(columns={'dates':'date', 'titles':'title','contents':'content'})
df['content'] = df['title'] + df['content']

In [4]:
df = df.drop(columns=['근거', 'title'])

In [5]:
df['label'] = df['label'].astype(int)
df

,date,content,label
0,2021.10.05,[[톡톡 이상품] 한빛코리아 / 다지워 네이처 클렌징 솝]//[\n마스카라까지 지우...,1
1,2021.10.07,"[[격동의 유통가④]편의점 성장 정체…미래는 근거리 쇼핑]//[""파괴적 커머스 시대...",1
2,2021.10.08,[株머니 매니저의 HOT종목]//[\n\n\n\n\n 씨에스윈드는 풍력발전기를 구성...,1
3,2021.10.11,"[김민수 더맘마 대표 ""동네마트와 '상생'이 경쟁력…30분 총알배송 목표""]//[\...",1
4,2021.10.13,"[""카카오 보고있나""…진격의 네이버페이, 中 최대 직구몰 손잡았다]//[[주간투자동...",1
...,...,...,...
274,2022.09.23,"[WCP 흥행 실패에 컬리, 케이뱅크 상장 시기 '안갯속']//[\n심사 승인 났는...",0
275,2022.09.26,"[[먹거리+IT] 네이처랩 오경주 대표, “맛있고 건강한 주스 추천 서비스를 개발합...",1
276,2022.09.27,"[""투자 늘렸는데 이제와서…"" 퀵커머스마저 규제하나 '초긴장']//[\n■국감 오르...",0
277,2022.09.29,[국내 이커머스 상장 1호 주인공 누가될까 [격변의 이커머스①]]//[\n이커머스 ...,0


### 데이터 정제하기

In [6]:
# 문자 외의 것들 제거
df['content'] = df['content'].str.replace('[^가-힣a-zA-Z0-9_]',' ')
# 혹시 모를 다중 공백 제거
df['content'] = df['content'].str.replace(' +',' ') 
df

/var/folders/1d/lfkpl6ps02xgh0rdprtbmgvw0000gn/T/ipykernel_2223/2956849729.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^가-힣a-zA-Z0-9_]',' ')
/var/folders/1d/lfkpl6ps02xgh0rdprtbmgvw0000gn/T/ipykernel_2223/2956849729.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace(' +',' ')


,date,content,label
0,2021.10.05,톡톡 이상품 한빛코리아 다지워 네이처 클렌징 솝 마스카라까지 지우는 세정비누 피부...,1
1,2021.10.07,격동의 유통가 편의점 성장 정체 미래는 근거리 쇼핑 파괴적 커머스 시대 데이터 경...,1
2,2021.10.08,머니 매니저의 HOT종목 씨에스윈드는 풍력발전기를 구성하는 타워 제작 및 설치를 ...,1
3,2021.10.11,김민수 더맘마 대표 동네마트와 상생 이 경쟁력 30분 총알배송 목표 별도 물류센터...,1
4,2021.10.13,카카오 보고있나 진격의 네이버페이 최대 직구몰 손잡았다 주간투자동향 파운트 400...,1
...,...,...,...
274,2022.09.23,WCP 흥행 실패에 컬리 케이뱅크 상장 시기 안갯속 심사 승인 났는데 IPO 감감...,0
275,2022.09.26,먹거리 IT 네이처랩 오경주 대표 맛있고 건강한 주스 추천 서비스를 개발합니다 서...,1
276,2022.09.27,투자 늘렸는데 이제와서 퀵커머스마저 규제하나 초긴장 국감 오르는 유통업 새 먹거리...,0
277,2022.09.29,국내 이커머스 상장 1호 주인공 누가될까 격변의 이커머스 이커머스 IPO 대전 흑...,0


In [7]:
train = df

In [8]:
train, test = train_test_split(train)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해주었습니다.
test.reset_index(inplace=True)

In [9]:
train = train.drop(['index'],axis=1)

In [10]:
train

,date,content,label
0,2022.05.16,대어도 잇단 무산 IPO 혹한기 상장 전략 비상 SK쉴더스 원스토어 상장 철회쏘카...,0
1,2022.09.04,식품 가전 특화 e커머스 결국 종합몰로 간다 무한 경쟁에 종합몰화 외형 확대 통한...,1
2,2022.01.07,위기가 기회 집콕 일상 돕는 모바일 앱 전성시대 2022년 전망 모바일 장보기 운...,1
3,2021.12.11,2021 이커머스 업계 합종연횡 새벽배송 기업 상장 앞으로 2021 결산 새 주인...,1
4,2022.05.31,오아시스마켓 맛집 그대로 세상 맛집들의 모임 이젠 빵집까지 간편식의 세계 K OT...,1
...,...,...,...
204,2021.10.08,머니 매니저의 HOT종목 씨에스윈드는 풍력발전기를 구성하는 타워 제작 및 설치를 ...,1
205,2022.04.20,승자 없는 새벽배송 전쟁 그 결말은 투자비 매출 롯데온 헬로네이처 철수대기업 위주...,0
206,2022.09.29,국내 이커머스 상장 1호 주인공 누가될까 격변의 이커머스 이커머스 IPO 대전 흑...,0
207,2022.06.10,제2의 루나사태 막는다 코인마켓 거래소들 공동 가이드라인 첫삽 한국디지털자산사업자...,0


In [11]:
test = test.drop(['index'],axis=1)#.drop(['index'],axis=1) 

In [12]:
test

,date,content,label
0,2022.04.06,딱 하나만 파는 버티컬커머스 전문성 무기로 소비자 마음 잡았다 코로나 특수 유통업...,1
1,2022.09.08,기업가치 1 오아시스마켓 상장 예심 청구 올해 IPO 마켓컬리 이어 두번째 식품 ...,1
2,2021.12.15,각자도생 살길 찾는 1세대 이커머스 미스터아빠 데이웰즈 경북프레쉬 센터 설립 체결...,1
3,2022.06.09,점유율 뺏길까 새벽배송에 뛰어드는 유통 업계 IPO 걸림돌 될라 구주매출 없애는 ...,0
4,2021.12.10,오아시스마켓 통합물류센터 힘입어 새벽배송 유일 흑자 2021 스타브랜드 대상 위기...,1
...,...,...,...
65,2022.04.12,새벽배송 1호 컬리 목표 시총 6조지만 거품 논란도 케이뱅크 MZ세대 맞춤형 케이...,0
66,2021.10.18,네이버 카카오 3분기 맑음 규제 회초리에도 성장 멈추지 않아 각자의 길 정한 이커...,1
67,2021.10.21,오아시스부터 컬리까지 이커머스 업계 대세된 렌털 난각번호 1번 계란 주세요 비싸도...,1
68,2022.04.17,대기업도 출혈에 포기 새벽배송 재편 신호탄 임세령은 왜 정육각을 pick했나 대상...,0


from sklearn.model_selection import train_test_split
X = df['content']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

X_train

X_test

### 토큰화 및 TF-IDF(Term Frequency-Inverse Document Frequency) 벡터화

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

from sklearn.linear_model import LinearRegression

In [14]:
test= test.drop(['label'],axis=1)

In [15]:
test

,date,content
0,2022.04.06,딱 하나만 파는 버티컬커머스 전문성 무기로 소비자 마음 잡았다 코로나 특수 유통업...
1,2022.09.08,기업가치 1 오아시스마켓 상장 예심 청구 올해 IPO 마켓컬리 이어 두번째 식품 ...
2,2021.12.15,각자도생 살길 찾는 1세대 이커머스 미스터아빠 데이웰즈 경북프레쉬 센터 설립 체결...
3,2022.06.09,점유율 뺏길까 새벽배송에 뛰어드는 유통 업계 IPO 걸림돌 될라 구주매출 없애는 ...
4,2021.12.10,오아시스마켓 통합물류센터 힘입어 새벽배송 유일 흑자 2021 스타브랜드 대상 위기...
...,...,...
65,2022.04.12,새벽배송 1호 컬리 목표 시총 6조지만 거품 논란도 케이뱅크 MZ세대 맞춤형 케이...
66,2021.10.18,네이버 카카오 3분기 맑음 규제 회초리에도 성장 멈추지 않아 각자의 길 정한 이커...
67,2021.10.21,오아시스부터 컬리까지 이커머스 업계 대세된 렌털 난각번호 1번 계란 주세요 비싸도...
68,2022.04.17,대기업도 출혈에 포기 새벽배송 재편 신호탄 임세령은 왜 정육각을 pick했나 대상...


In [16]:
from sklearn.metrics import f1_score

In [17]:
max_features = 3000

vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features)

X = train['content']
y = np.array(train.label)

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)

# evals 변수 생성
vals = [(X_val, y_val)]

# lin_reg = LinearRegression()
lgs = LogisticRegression(class_weight='balanced') # 

# lin_reg.fit(X_train, y_train)
lgs.fit(X_train, y_train)

# predicted = lin_reg.predict(X_val)
predicted = lgs.predict(X_val)

#     print(f"Linear Regression      max_features: {max_features}, Accuracy: {lin_reg.score(X_val, y_val)}" )
print(f"Logistic Regression      max_features: {max_features}, Accuracy: {lgs.score(X_val, y_val)}" )
# print(f1_score())


Logistic Regression      max_features: 3000, Accuracy: 0.7358490566037735


In [ ]:
lgs.get_params()

In [ ]:
y_val.size

In [18]:
# Scikit-learn Wrapper XGBoost 클래스인 XGBClassifier 임포트
from xgboost import XGBClassifier

# evals 변수 생성
vals = [(X_val, y_val)]

# XGBClassifier 클래스의 객체 생성
# xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=5)
# xgb_wrapper = XGBClassifier(learning_rate= 0.1, max_depth= 3, n_estimators= 200)

# 모델 학습
# xgb_wrapper.fit(X_train, y_train, early_stopping_rounds=400, eval_set=vals, eval_metric="logloss",  verbose=True)

# 예측
# pred = xgb_wrapper.predict(X_val)

# 평가 - 정확도 확인
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, pred)
print('인공 신경망 예측 정확도: {0:.4f}'.format(accuracy), '\n')


NameError: name 'xgb_wrapper' is not defined

In [19]:
from sklearn.model_selection import GridSearchCV

params = {
    'C': [0.1, 0.01, 0.001],
    'max_iter' : [3, 5, 7],
    'n_jobs' : [200, 400, 600]
}

grid_cv = GridSearchCV(lgs, param_grid=params, scoring='accuracy', cv=3, verbose=1)
grid_cv.fit(X_train, y_train, eval_set=vals, eval_metric="logloss", verbose=True)
print('GridSearchCV 최고 평균 정확도 수치:{0:.4f}'.format(grid_cv.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv.best_params_)


Fitting 3 folds for each of 27 candidates, totalling 81 fits


ValueError: 
All the 81 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
81 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/lhs/miniforge3/envs/lhs/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
TypeError: fit() got an unexpected keyword argument 'eval_set'


In [ ]:
grid_cv.get_params()

In [ ]:
for max_features in range(100, 1000, 100):
    vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features)

    X = train['content']
    y = np.array(train.label)
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)
    
    X_train = vectorizer.fit_transform(X_train)
    X_val = vectorizer.transform(X_val)
    
    lin_reg = LinearRegression()
    lgs = LogisticRegression(class_weight='balanced') # 
    
    lin_reg.fit(X_train, y_train)
    lgs.fit(X_train, y_train)
    
    predicted = lin_reg.predict(X_val)
    predicted = lgs.predict(X_val)
    
#     print(f"Linear Regression      max_features: {max_features}, Accuracy: {lin_reg.score(X_val, y_val)}" )
    print(f"Logistic Regression      max_features: {max_features}, Accuracy: {lgs.score(X_val, y_val)}" )
    print(f1_score(X_val, y_val, average='macro'))
    

In [ ]:
from sklearn.metrics import mean_squared_error

y_train = lin_reg.predict(y_test)

In [ ]:
# tf-idf 벡터화
for max_features in range(100, 10000, 100):
    vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features)

    X = train['content']
    y = np.array(train.label)

    X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=2)

    X_train = vectorizer.fit_transform(X_train)
    X_eval = vectorizer.transform(X_eval)
    
    lgs = LogisticRegression(class_weight='balanced')
    naive = BernoulliNB()
    sgd = SGDClassifier(random_state=42, n_jobs=-1)
    rfc = RandomForestClassifier(random_state=42, n_jobs=-1)
    svc = SVC(random_state=42)
    ada = AdaBoostClassifier(random_state=42)
    
    lgs.fit(X_train, y_train)
    naive.fit(X_train, y_train)
    sgd.fit(X_train, y_train)
    rfc.fit(X_train, y_train)
    svc.fit(X_train, y_train)
    ada.fit(X_train, y_train)
    
    predicted = lgs.predict(X_eval)
    predicted = naive.predict(X_eval)
    predicted = sgd.predict(X_eval)
    predicted = rfc.predict(X_eval)
    predicted = svc.predict(X_eval)
    predicted = ada.predict(X_eval)
    
    print(f"1. Logistic_Regression      max_features: {max_features}, Accuracy: {lgs.score(X_eval, y_eval)}" )
    print(f"2. Naive_bayse             max_features: {max_features}, Accuracy: {naive.score(X_eval, y_eval)}" )
    print(f"3. SGD                     max_features: {max_features}, Accuracy: {sgd.score(X_eval, y_eval)}" )
    print(f"4. RandomForestClassifier  max_features: {max_features}, Accuracy: {rfc.score(X_eval, y_eval)}" )
    print(f"5. SVC                     max_features: {max_features}, Accuracy: {svc.score(X_eval, y_eval)}" )
    print(f"6. AdaBoost                max_features: {max_features}, Accuracy: {ada.score(X_eval, y_eval)}" )
    print('=======')

In [ ]:
from xgboost import XGBClassifier

X = train['content']
y = np.array(train.label)

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=2)

X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)

# XGBClassifier 클래스의 객체 생성
xgb = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=5)

# 모델 학습
xgb.fit(X_train, y_train, early_stopping_rounds=400, eval_set=vals, eval_metric="logloss",  verbose=True)

# 예측
predicted = xgb_wrapper.predict(X_val)

# 평가 - 정확도 확인
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, pred)
print('인공 신경망 예측 정확도: {0:.4f}'.format(accuracy))


In [ ]:
# 2600 - 3000 이 0.83으로 정확도가 가장 높게 나옴

vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=2600)

X = train['content']
y = np.array(train.label)

X_train = vectorizer.fit_transform(X)
X_test = vectorizer.transform(test['content'])
lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y)
predicted = lgs.predict(X_test)

X_train = vectorizer.fit_transform(train['content'])
X_test = vectorizer.transform(test['content'])
y = np.array(train.label)

# lgs = LogisticRegression(class_weight='balanced')
# lgs.fit(X_train, y)
# predicted = lgs.predict(X_test)
rfc = RandomForestClassifier(random_state=42, n_jobs=-1)
rfc.fit(X_train, y)
predicted = rfc.predict(X_test)

In [ ]:
predicted

In [ ]:
a = predicted.tolist()
b = np.array(test.label).tolist()

In [ ]:
c = [i for i,j in zip(a,b) if i == j]
print(len(c))

In [ ]:
correct_ratio = len(c) / len(test) * 100
correct_ratio

In [ ]:
news_feature = df.drop(columns = ['content'])

In [ ]:
news_feature.to_csv('news_feature.csv')